In [40]:
from gurobipy import *
import snap
import numpy as np
import copy
import random
import matplotlib.pyplot as plt
import time

In [41]:
#load a graph G
node = 13
# G = np.loadtxt('test_10nodes.txt', delimiter='\t', skiprows=3, dtype="int")
# G = np.loadtxt('facebook_combined.txt', delimiter=' ', dtype="int")
# G = G.tolist()

FIn = snap.TFIn("small_tree.graph")
G1 = snap.TNGraph.Load(FIn)

# select k nodes
k = 5

In [42]:
#build up internal opinion s
s = np.loadtxt('internal_opinion_small_tree.txt', dtype="float")
s = s.tolist()

In [43]:
W = [[0]*node for i in range(node)]
for NI in G1.Nodes():
    for Id in NI.GetOutEdges():
        W[NI.GetId()][Id]=1

V = []
for i in range(node):
    V.append([])
for i in range(len(W)):
    for j in range(len(W[i])):
        if W[i][j]==1:
            V[i].append(W[i].index(W[i][j]))
print(V)

[[], [0], [0], [0], [1], [1], [1], [2], [2], [2], [3], [3], [3]]


In [44]:
#create blank model
LP_model = Model('Linear Program')

#add variables
Z = LP_model.addVars([(i) for i in range(node)],lb=0, ub=1, name="Z")
Y = LP_model.addVars([(i) for i in range(node)] ,lb=0, ub=1, name="Y")
X = LP_model.addVars([(i) for i in range(node)],lb=0, ub=1, name="X")
LP_model.update()

#add objective number
LP_model.setObjective(quicksum(Z[i] for i in range(node)), GRB.MAXIMIZE)

#constraint 1
LP_model.addConstr(quicksum(Y[i] for i in range(node)) <= k)
#constraint 2
LP_model.addConstr(Z[0]==Y[0]+s[0]-s[0]*Y[0])
#constraint 3
# LP_model.addConstrs(Z[i] == Y[i]+(s[i]+Z[j])/2-(s[i]*Y[i])/2-X[i]/2 for i in range(1,node) for j in V[i])
LP_model.addConstrs(Z[i] <= Y[i]+(s[i]+Z[j])/2-(s[i]*Y[i])/2-X[i]/2 for i in range(1,node) for j in V[i])
#constraint 4
LP_model.addConstrs(X[i]<=Z[j] for i in range(1,node) for j in V[i])

# LP_model.addConstrs(Y[i]<=Z[i] for i in range(1,node))

LP_model.update()

LP_model.setParam("presolve",0)
LP_model.setParam('SubMIPNodes',0)
LP_model.setParam('Cuts',2)
LP_model.setParam('MIRCuts',0)
LP_model.setParam('Threads',1)
start = time.perf_counter()
LP_model.optimize()
sub_end = time.perf_counter()
print('time:', sub_end-start)

Changed value of parameter presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter SubMIPNodes to 0
   Prev: 500  Min: 0  Max: 2000000000  Default: 500
Changed value of parameter Cuts to 2
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter MIRCuts to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Threads to 1
   Prev: 0  Min: 0  Max: 1024  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads
Optimize a model with 26 rows, 39 columns and 87 nonzeros
Model fingerprint: 0x7331c85c
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-02, 5e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3000000e+01   2.899810e+00   0.000000e+00      0s
      12    1.3000000e+01   0.000000e+00   0.000000e+00      0s

Solved

In [45]:
# print(LP_model.ObjVal)
# for var in LP_model.getVars():
#     print(var.varName,'\t',var.x)

In [46]:
def equilibrium(z, W):
    z_prev = copy.deepcopy(z)
    cost = [0 for n in range(node)] #c_i(z)
    cost_prev = [0 for n in range(node)] 

    max_round = 10000

    for i in range(max_round):
        
        for NI in G1.Nodes():
            temp_zi_zj_sum = 0   #sum(w_ij(z_i-z_j)^2) ==> 3.1
            temp_z_neighbor = 0   #sum(w_ij z_j) ==> 3.2
            temp_neighbor_weight = 0  #sum(w_ij) ==>3.2

            for Id in NI.GetOutEdges():
                temp_zi_zj_sum += W[NI.GetId()][Id]*(z[NI.GetId()]-z[Id])**2 #3.1
                temp_z_neighbor += W[NI.GetId()][Id]*z_prev[Id] #3.2
                temp_neighbor_weight += W[NI.GetId()][Id] #3.2
                
            cost[NI.GetId()] = (s[NI.GetId()]-z[NI.GetId()])**2 + temp_zi_zj_sum #3.1

            if z[NI.GetId()]==1:
                z[NI.GetId()] = 1
            else:
                z[NI.GetId()] = (s[NI.GetId()]+temp_z_neighbor)/(1+temp_neighbor_weight) #3.2
#         print(z)
        if abs(sum(cost_prev)-sum(cost))<0.00000000000001:
            break

        cost_prev = copy.deepcopy(cost)
        z_prev = copy.deepcopy(z)
    return z

In [47]:
y = []
for i in range(node):
    y.append(i)

# sum_LP_solve = []    
# for j in range(1000): 
#     selected = []
#     all_vars = []
#     all_bar_y =  []
#     for var in LP_model.getVars():
#         all_vars.append(var.x)
#     for i in range(node, node*2):
#         all_bar_y.append(all_vars[i])
#     for i in range(k):
#         select_v = random.choices(y, weights = all_bar_y, k=1)
#     #     print(select_v[0])
#     #     y[select_v[0]]=-1
#         all_bar_y[select_v[0]]=0
#         selected.append(select_v[0])
# #     print(selected)
#     z = copy.deepcopy(s)
#     for i in range(len(selected)):
#         z[selected[i]] = 1
#     LP_solve = sum(equilibrium(z, W))
# #     print(LP_solve)
#     sum_LP_solve.append(LP_solve)
# print(sum(sum_LP_solve)/len(sum_LP_solve))

In [48]:
selected = []
all_vars = []
all_bar_y =  []
for var in LP_model.getVars():
    all_vars.append(var.x)
for i in range(node, node*2):
    all_bar_y.append(all_vars[i])
for i in range(k):
    select_v = random.choices(y, weights = all_bar_y, k=1)
#     print(select_v[0])
#     y[select_v[0]]=-1
    all_bar_y[select_v[0]]=0
    selected.append(select_v[0])
#     print(selected)
z = copy.deepcopy(s)
for i in range(len(selected)):
    z[selected[i]] = 1
equilibrium(z, W)
end = time.perf_counter()
print('time: ',end - start)

time:  0.07839770001010038


In [49]:
print(10.935825325887178/11.894395930949573) #small

0.9194098960025229


In [50]:
print(520.6121731840166/558.8729695901077) #large

0.9315393685363731


In [51]:
print(558.2954466648072/558.8729695901077)

0.9989666293474095


In [52]:
print(365.3755578392006/365.7031671582477)

0.999104166032816
